In [13]:
import os

import pandas as pd

In [14]:
os.environ["IMAGEIO_FFMPEG_EXE"] = "/Users/user/Documents/GitHub/Video-from-Script/ffmpeg"
from moviepy.editor import *

In [24]:
def need_video(df_loc):

    df = pd.read_excel(df_loc)
    
    need_video = df[(df.Script_Created == "Yes") &
                    (df.AI_Voice_Generated == "Yes") &
                    (df.Images_Obtained == "Yes") &
                    (df.Youtube_Video_Created == "No")]

    return need_video

In [25]:
def create_video(figure, audio_loc, image_loc, video_loc, df_loc, yt_short=False):
    
    # Get the locations
    audio_location = audio_loc + figure + '.mp3'
    image_location = image_loc + figure + '/'
    video_location = video_loc + figure + '.mp4'
        
    # Add audio clip and get the duration
    audio_clip = AudioFileClip(audio_location)
    audio_duration = audio_clip.duration
        
    # Get the time each image will show for, remove thumbnail options
    possible_images = os.listdir(image_location)
    possible_images = [image for image in possible_images if "thumbnail" not in image]
    possible_images = [ i for i in possible_images if i != '.DS_Store']
        
    number_of_images = len(possible_images)
    image_duration = audio_duration / number_of_images

    # Here we create the frames for each image and their duration
    img_names = possible_images
    img = list(image_location + img for img in img_names)

    clips = [ImageClip(m).set_duration(image_duration) for m in img]

    # Combine the images together into a single clip
    concat_clip = concatenate_videoclips(clips, method = "compose")

    # add the audio
    concat_clip.audio = audio_clip

    # Write the video with audio included
    concat_clip.write_videofile(video_location, fps = 24)

    # Add Yes to the video created column
    df = pd.read_excel(df_loc)
    df.loc[(df.Name == figure), "Youtube_Video_Created"] = "Yes"
    df.to_excel(df_loc, index=False)